In [25]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [26]:
loader = PyPDFLoader("YogiMeterFeeler222F(32.1).pdf")
docs = loader.load_and_split()
docs    

[Document(metadata={'source': 'YogiMeterFeeler222F(32.1).pdf', 'page': 0}, page_content='73.2% 26.8%\nFEELING/THINKING SOLVING/LEARNING\nYogiMeter analyses your Choice Making Mind and projects your decision making according to\nyour current Mindset. Learn to understand and change your mindset to become your best self.\nCAS (Cognitive Attention Score)\nCurrently your feeling mind is running your life. You are either spoilt for choices or just slow and confused. Your\nmind makes you spend too much time in the future. Your mind is playing tricks on you. It does not let you focus\nand does not let you progress or �nd peace. Your focus is also compromised.'),
 Document(metadata={'source': 'YogiMeterFeeler222F(32.1).pdf', 'page': 2}, page_content='1 days\n2 days\n32 days\n18 days\n7 days\nYogiMeter has detected your general and leadership mental wellness scores.\nUpcoming 60 days'),
 Document(metadata={'source': 'YogiMeterFeeler222F(32.1).pdf', 'page': 3}, page_content='Sometimes your mind i

In [27]:
template = """
write a concise and short summary of the following speech:
Speech: {text}
"""

prompt = PromptTemplate( input_variables=["text"] ,template=template)
prompt

PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\nwrite a concise and short summary of the following speech:\nSpeech: {text}\n')

In [28]:
from langchain.chains.summarize import load_summarize_chain
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")

In [29]:
llm=ChatGroq(groq_api_key=groq_api_key, model="gemma2-9b-it")

In [52]:
chain = load_summarize_chain(llm=llm, chain_type="stuff", prompt=prompt, verbose=True)
response = chain.run(docs)
response

ValidationError: 1 validation error for RefineDocumentsChain
prompt
  Extra inputs are not permitted [type=extra_forbidden, input_value=PromptTemplate(input_vari...ech:\nSpeech: {text}\n'), input_type=PromptTemplate]
    For further information visit https://errors.pydantic.dev/2.10/v/extra_forbidden

### Using map reduce to summerize documents

In [42]:
final_docs=RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=100).split_documents(docs)
len(final_docs)

71

In [44]:
chunks_prompt = """
Please summerize the below speech:
Speech: {text}
Summary:
"""

map_prompt_temp = PromptTemplate(input_variables=["text"], template=chunks_prompt)
map_prompt_temp

PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\nPlease summerize the below speech:\nSpeech: {text}\nSummary:\n')

In [47]:
final_prompt = """
Provide the final summary of entire speech with these importance ponits:
Add a motivation title, start and precise summary with an introduction and provide the summary in number of points of the speech.

speech: {text}
"""

final_prompt = PromptTemplate(input_variables=["text"], template=final_prompt)

In [51]:
summary_chain = load_summarize_chain(
    llm=llm,
    chain_type="map_reduce",
    map_prompt=map_prompt_temp,
    combine_prompt=final_prompt,
    verbose=True
)
summary_chain

MapReduceDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\nPlease summerize the below speech:\nSpeech: {text}\nSummary:\n'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7f79949d2b90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7f79947339a0>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********')), output_parser=StrOutputParser(), llm_kwargs={}), reduce_documents_chain=ReduceDocumentsChain(verbose=True, combine_documents_chain=StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\nProvide the final summary of entire speech with these importance ponits:\nAdd a motivation title, start and precise summary with an introduction and provide the summary in number of points of the speech.\

In [49]:
output = summary_chain.run(final_docs)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Please summerize the below speech:
Speech: 73.2% 26.8%
FEELING/THINKING SOLVING/LEARNING
YogiMeter analyses your Choice Making Mind and projects your decision making according to
your current Mindset. Learn to understand and change your mindset to become your best self.
CAS (Cognitive Attention Score)
Currently your feeling mind is running your life. You are either spoilt for choices or just slow and confused. Your
mind makes you spend too much time in the future. Your mind is playing tricks on you. It does not let you focus
and does not let you progress or �nd peace. Your focus is also compromised.
Summary:

Prompt after formatting:

Please summerize the below speech:
Speech: 1 days
2 days
32 days
18 days
7 days
YogiMeter has detected your general and leadership mental wellness scores.
Upcoming 60 days
Summary:

Prompt after formatting:

Please summerize the below speech:
Spe

/var/www/learning/genAIApps/langchain-framework/langenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Token indices sequence length is longer than the specified maximum sequence length for this model (10907 > 1024). Running this sequence through the model will result in indexing errors




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Provide the final summary of entire speech with these importance ponits:
Add a motivation title, start and precise summary with an introduction and provide the summary in number of points of the speech.

speech: This speech excerpt promotes YogiMeter, a tool that analyzes your "Choice Making Mind" and predicts your decisions based on your current mindset.  

It suggests that you're currently dominated by your "feeling mind," leading to indecisiveness, overthinking the future, and difficulty focusing. This state prevents progress and peace.  YogiMeter aims to help you understand and shift your mindset for better decision-making and overall well-being. 


The speech provides a snapshot of mental wellness scores tracked by YogiMeter over various time periods. 

Here's a breakdown:

* **Past Scores:**
    * 1 day:  Score not specified
    * 2 days: Score not specified
    * 32 days: S

In [54]:
chain = load_summarize_chain(llm=llm, chain_type="refine", refine_prompt=prompt, verbose=True)
response = chain.run(final_docs)
response



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"73.2% 26.8%
FEELING/THINKING SOLVING/LEARNING
YogiMeter analyses your Choice Making Mind and projects your decision making according to
your current Mindset. Learn to understand and change your mindset to become your best self.
CAS (Cognitive Attention Score)
Currently your feeling mind is running your life. You are either spoilt for choices or just slow and confused. Your
mind makes you spend too much time in the future. Your mind is playing tricks on you. It does not let you focus
and does not let you progress or �nd peace. Your focus is also compromised."


CONCISE SUMMARY:

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

write a concise and short summary of the following speech:
Speech: 1 days
2 days
32 days
18 days
7 days
YogiMeter has detected your general and leadership mental wellness scores.
Upcom

'This appears to be the footer of a website, listing basic navigational links (Help, About, FAQ, Contact, Terms, Privacy) and copyright information for the year 2024, belonging to "Ceekr." \n'